---
title: ♾️伪随机数发生器
password: ""
tags:
  - 密码学
katex: false
comments: true
aside: true
date: 2022-07-01 00:59:59
cover: https://www.helloimg.com/images/2022/07/01/ZMd92o.jpg
top_img:
---

<!--
 * @?: *********************************************************************
 * @Author: Weidows
 * @LastEditors: Weidows
 * @LastEditTime: 2022-04-20 23:11:24
 * @FilePath: \Blog-private\scaffolds\post.md
 * @Description:
 * @!: *********************************************************************
-->

## 序

此发生器基于空间数组, 随机种子来自于 time 时间戳末尾位

随着 random 次数增加, array 不确定度持续`叠加`, 每位概率会有一定波动但不会出现偏倚或黑洞

熵来自于每次 update 的位置和次序, 淡化了某时刻 timestamp 和算力的影响

思想类似卷积, 把过去与当前状态持续叠加到对未来的影响中

<a>![分割线](https://www.helloimg.com/images/2022/07/01/ZM0SoX.png)</a>

## 代码


In [6]:
from datetime import datetime

array = [i for i in range(10)]


def random():
    time_str = str(datetime.now().timestamp())
    if len(time_str) == 16: index = 0
    else:
        index = int(time_str[-1])

    # core: 每次 random, 使 index 位加上下一位的数字
    array[index] = (array[index] + array[(1 + index) % len(array)]) % 10

    return array[index]


def measure(random_times):
    times = [0 for _ in range(10)]
    for i in range(random_times):
        num = random()
        times[num] += 1

        random_times_div_10 = random_times // 10
        if i % random_times_div_10 == 0:
            for i in range(10):
                times[i] = times[i] / random_times_div_10
            print(times)
            times = [0 for _ in range(10)]


if __name__ == '__main__':
    # print(random())

    # 数量太小, 算力太大把随机度给淹了
    print("1K 个随机数 0-9 平均概率:")
    measure(1_000)

    print()

    print("1M 个随机数 0-9 平均概率:")
    measure(1_000_000)


1K 个随机数 0-9 平均概率:
[0.0, 0.0, 0.0, 0.01, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]

1M 个随机数 0-9 平均概率:
[0.0, 0.0, 0.0, 1e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.08707, 0.09659, 0.07117, 0.09658, 0.08152, 0.11249, 0.08783, 0.15348, 0.07114, 0.14213]
[0.07329, 0.09794, 0.08054, 0.11958, 0.08392, 0.09796, 0.07334, 0.10519, 0.08776, 0.18048]
[0.11594, 0.10057, 0.09201, 0.12211, 0.05934, 0.10781, 0.05929, 0.10423, 0.08083, 0.15787]
[0.11348, 0.09361, 0.09942, 0.09361, 0.08974, 0.11739, 0.1132, 0.09622, 0.08972, 0.09361]
[0.09697, 0.08811, 0.07557, 0.13141,

<a>![分割线](https://www.helloimg.com/images/2022/07/01/ZM0SoX.png)</a>

## 现有缺陷

1. 无法保证 `恒定的概率` 或者说 `等概率随机`, 但可以近似做到 `自然随机`
2. array 需要预热, 预热效果也会影响一定范围内的随机数质量
3. 小批量随机效果贼差

## 借物表

暂无.
